In [104]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [105]:
from gensim import models,corpora,similarities
import jieba.posseg as pseg
from gensim.models.doc2vec import TaggedDocument,Doc2Vec
import os

In [106]:
def a_sub_b(a,b):
    ret = []
    for el in a:
        if el not in b:
            ret.append(el)
    return ret
stop = [line.strip() for line in open('../input/word2vec-test/stop_words.txt','r',encoding='utf-8').readlines() ]

In [107]:
#读取文件
raw_documents=[]
for root, dirs, files in os.walk(os.path.realpath("/kaggle/input/train-test")):
    for name in files:
        print(os.path.join(root,name))
        try:
            f = open(os.path.join(root, name), 'r',encoding = 'gb18030')
            raw = f.read()
            raw_documents.append(raw)
        except:
            f = open(os.path.join(root, name), 'r',encoding = 'utf-8')
            raw = f.read()
            raw_documents.append(raw)

/kaggle/input/train-test/hongloumeng.txt
/kaggle/input/train-test/xusanguanmaixueji.txt
/kaggle/input/train-test/hetongjiufen.txt
/kaggle/input/train-test/huozhe.txt
/kaggle/input/train-test/zhengfugongzuobaogao.txt
/kaggle/input/train-test/fengkuangxingqisi.txt
/kaggle/input/train-test/xinwen.txt
/kaggle/input/train-test/sharen.txt


In [108]:
#构建语料库
corpora_documents = []
doc=[]            #输出时使用，用来存储未经过TaggedDocument处理的数据，如果输出document，前面会有u
for i, item_text in enumerate(raw_documents):
    print(i)
    words_list=[]
    item=(pseg.cut(item_text))
    for j in list(item):
        words_list.append(j.word)
    words_list=a_sub_b(words_list,list(stop))
    document = TaggedDocument(words=words_list, tags=[i])
    corpora_documents.append(document)
    doc.append(words_list)

0
1
2
3
4
5
6
7


In [109]:
#创建model
model = Doc2Vec(vector_size=13, min_count=1, window=5)
model.build_vocab(corpora_documents)
model.train(corpora_documents,total_examples=model.corpus_count,epochs=200)
print(model)

Doc2Vec<dm/m,d13,n5,w5,s0.001,t3>


In [110]:
def str2vec(str):
    test_cut = []
    item = (pseg.cut(str))
    for i in list(item):
        test_cut.append(i.word)
    test_cut = a_sub_b(test_cut,list(stop))
    inferred_vector = model.infer_vector(test_cut)
    return inferred_vector

In [111]:
file_name = []
file_vec = []
for root, dirs, files in os.walk(os.path.realpath("/kaggle/input/doc2vec-test")):
    for name in files:
        print(os.path.join(root,name))
        f = open(os.path.join(root, name), 'r',encoding = 'utf-8')
        raw = f.read()
        temp = str2vec(raw)
        print(temp)
        file_vec.append(temp)
        file_name.append(name)

/kaggle/input/doc2vec-test/xusanguanA.txt
[-6.115202  -1.6067901 -1.18203    7.5624933  5.2422123 -4.474884
 -7.351678  -5.4728637  3.467514   1.1448207  1.044417  -0.3009571
 -3.6676137]
/kaggle/input/doc2vec-test/1989.txt
[-3.6507978  -4.06832    -4.4141555   0.68011844  9.78004     8.599002
 -8.1373825  -4.205593   -0.41213503 10.291586   -1.0825031  -0.53131
 -1.6170894 ]
/kaggle/input/doc2vec-test/huozheA.txt
[ 2.366646   -3.4371765  -1.767743    6.0021048  -0.44266483 -1.6350086
 -5.8339105  -6.859003    1.5538522   3.9193888  -0.3826901  -2.9032843
 -5.1711383 ]
/kaggle/input/doc2vec-test/sharenB.txt
[ -6.5505304 -15.734934  -11.96612     1.6371117  27.143587    6.1281347
 -24.429485   -8.906998   -7.446186   -2.7314231   3.6510632  -6.0285993
 -11.061704 ]
/kaggle/input/doc2vec-test/hetongB.txt
[-5.409726   -3.008277    0.7471772  -2.5670002   4.9130726   4.174725
 -1.2767237  -6.6969414  -4.879846   -0.0396223  -0.46070853 -1.6854951
 -4.1064243 ]
/kaggle/input/doc2vec-test/xu

In [112]:
temp_vec = []
for i in range(len(file_name)):
    file_name[i] = file_name[i][:-4] + "_vec.txt"
    temp_vec.append([float('{:.6f}'.format(j)) for j in file_vec[i].tolist()])
    with open(file_name[i],'w',encoding="utf-8") as f:
        f.write(str(temp_vec[i]))

In [113]:
file_len = len(file_vec)
print(file_len)
for i in range (file_len):
    for j in range (file_len):
        if(i!=j):
            print(file_name[i],file_name[j],np.linalg.norm(file_vec[i] - file_vec[j]))

12
xusanguanA_vec.txt 1989_vec.txt 19.266144
xusanguanA_vec.txt huozheA_vec.txt 12.046049
xusanguanA_vec.txt sharenB_vec.txt 38.407642
xusanguanA_vec.txt hetongB_vec.txt 17.250336
xusanguanA_vec.txt xusanguanB_vec.txt 7.7752056
xusanguanA_vec.txt 1988_vec.txt 17.714947
xusanguanA_vec.txt huozheB_vec.txt 11.896273
xusanguanA_vec.txt sharenA_vec.txt 41.0205
xusanguanA_vec.txt hongloumengB_vec.txt 15.714084
xusanguanA_vec.txt hongloumengA_vec.txt 16.363256
xusanguanA_vec.txt hetongA_vec.txt 22.420902
1989_vec.txt xusanguanA_vec.txt 19.266144
1989_vec.txt huozheA_vec.txt 18.887497
1989_vec.txt sharenB_vec.txt 34.036808
1989_vec.txt hetongB_vec.txt 16.509014
1989_vec.txt xusanguanB_vec.txt 16.291489
1989_vec.txt 1988_vec.txt 2.8007076
1989_vec.txt huozheB_vec.txt 18.394747
1989_vec.txt sharenA_vec.txt 36.556202
1989_vec.txt hongloumengB_vec.txt 19.603218
1989_vec.txt hongloumengA_vec.txt 18.738724
1989_vec.txt hetongA_vec.txt 18.158598
huozheA_vec.txt xusanguanA_vec.txt 12.046049
huozheA_ve

In [114]:
import os
import zipfile
import datetime

def file2zip(packagePath, zipPath):
    '''
  :param packagePath: 文件夹路径
  :param zipPath: 压缩包路径
  :return:
  '''
    zip = zipfile.ZipFile(zipPath, 'w', zipfile.ZIP_DEFLATED)
    for path, dirNames, fileNames in os.walk(packagePath):
        fpath = path.replace(packagePath, '')
        for name in fileNames:
            fullName = os.path.join(path, name)
            name = fpath + '\\' + name
            zip.write(fullName, name)
    zip.close()

In [115]:
packagePath = '/kaggle/working/'
zipPath = '/kaggle/working/output.zip'
if os.path.exists(zipPath):
    os.remove(zipPath)
file2zip(packagePath, zipPath)
print("打包完成")
print(datetime.datetime.utcnow())


打包完成
2023-05-09 06:39:32.978454


In [116]:
# import shutil
# import os
# path = '/kaggle/working/'
# if os.path.exists(path):
#     shutil.rmtree(path)
#     print('删除完成')
# else:
#     print('原本为空')